In [1]:
import os
os.getcwd()

'/work/TALC/enel645_2025w'

In [2]:
import os
os.listdir()

['garbage_data',
 'local_dataset',
 'transfer_learning',
 'slurm-35541.out',
 'best_model.pth',
 'CombinedModel.py',
 '.git',
 'cm1.slurm',
 'cm1.py',
 'jupyter_env',
 'Assignment2.slurm',
 '.ipynb_checkpoints',
 'test.ipynb']

In [3]:
import os
os.chdir('/work/TALC/enel645_2025w/garbage_data')

In [5]:
import os
os.listdir()

['CVPR_2024_dataset_Train',
 'CVPR_2024_dataset_Test',
 'CVPR_2024_dataset_Val',
 'Downloads',
 '.ipynb_checkpoints',
 'Data',
 'import os.py',
 'source',
 'garbage_data',
 'train.py',
 'Sampled_Train',
 'local_garbage_data',
 'Sampled_Val',
 'Sampled_Test',
 'garbage_data.tar.gz',
 'my_venv']

In [7]:
!pip install torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images
    transforms.ToTensor()           # Convert to Tensor
])

# Load dataset
dataset_path = "/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train"
train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Check a sample
image, label = train_dataset[0]
print(image.shape, label)

     |████████████████████████████████| 23.3 MB 2.7 MB/s            
     |████████████████████████████████| 881.9 MB 9.8 kB/s             
     |████████████████████████████████| 3.1 MB 50.4 MB/s            
     |████████████████████████████████| 14.8 MB 34.1 MB/s            
torch.Size([3, 224, 224]) 0


In [9]:
!pip install scikit-learn

     |████████████████████████████████| 22.2 MB 2.8 MB/s            
     |████████████████████████████████| 25.9 MB 49.4 MB/s            
     |████████████████████████████████| 309 kB 50.1 MB/s            


In [11]:
!pip install pandas

     |████████████████████████████████| 9.5 MB 3.0 MB/s            


In [17]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from PIL import Image
import os

In [20]:
import os
os.listdir()

['CVPR_2024_dataset_Train',
 'CVPR_2024_dataset_Test',
 'CVPR_2024_dataset_Val',
 'Downloads',
 '.ipynb_checkpoints',
 'Data',
 'import os.py',
 'source',
 'garbage_data',
 'train.py',
 'Sampled_Train',
 'local_garbage_data',
 'Sampled_Val',
 'Sampled_Test',
 'garbage_data.tar.gz',
 'my_venv']

In [18]:
class GarbageDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, self.data.iloc[idx, 0])
        image = Image.open(img_name).convert("RGB")
        text = self.data.iloc[idx, 1]  # Assuming the second column is textual data
        label = int(self.data.iloc[idx, 2])  # Assuming the third column is the label

        if self.transform:
            image = self.transform(image)

        return image, text, label

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = GarbageDataset("train.csv", "train_images", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [36]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Train", transform=transform)
val_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Val", transform=transform)
test_dataset = datasets.ImageFolder(root="/work/TALC/enel645_2025w/garbage_data/CVPR_2024_dataset_Test", transform=transform)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class mapping
print(train_dataset.class_to_idx)

{'Black': 0, 'Blue': 1, 'Green': 2, 'TTR': 3}


In [37]:
import torch.nn as nn
import torchvision.models as models

class GarbageClassifier(nn.Module):
    def __init__(self, num_classes=6):  # Adjust `num_classes` based on your dataset
        super(GarbageClassifier, self).__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = nn.Linear(self.cnn.fc.in_features, num_classes)

    def forward(self, image):
        return self.cnn(image)


In [ ]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GarbageClassifier(num_classes=len(train_dataset.classes)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
def train_model(model, train_loader, val_loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_acc = correct / total
        print(f"Epoch {epoch+1}: Loss={running_loss / len(train_loader)}, Accuracy={train_acc:.2f}")

train_model(model, train_loader, val_loader)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/mdtahmid.jamil/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100.0%


Epoch 1: Loss=0.9959615393654331, Accuracy=0.60


In [ ]:
#visualization and matrix
import matplotlib.pyplot as plt
import numpy as np

def visualize_errors(model, dataloader):
    model.eval()
    incorrect_samples = []

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for img, pred, true in zip(images, preds, labels):
                if pred != true:
                    incorrect_samples.append((img.cpu(), pred.cpu().item(), true.cpu().item()))

    # Show 5 incorrect samples
    fig, axs = plt.subplots(1, 5, figsize=(15, 3))
    for i, (img, pred, true) in enumerate(incorrect_samples[:5]):
        axs[i].imshow(img.permute(1, 2, 0))
        axs[i].set_title(f"Pred: {pred}, True: {true}")
        axs[i].axis("off")

    plt.show()

visualize_errors(model, test_loader)

In [ ]:
#save the train model
torch.save(model.state_dict(), "garbage_classifier.pth")